### 导包

In [102]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = 'all' #默认为'last'

import numpy as np
import pandas as pd
import os
import sympy 
# import numpy.linalg as LA
from sympy import *
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['font.sans-serif']=['SimHei'] #指定默认字体 SimHei为黑体
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
sympy.init_printing(use_latex=True)
x, y, z = symbols('x y z',real=True)

import torch
import scipy.linalg as la
import numpy.linalg as LA

np.set_printoptions(precision=2,suppress=True)

#Note that http urls will not be displayed. Only https are allowed inside the Iframe
from IPython.display import IFrame


# [图卷积神经网络(GCN)详解:包括了数学基础(傅里叶，拉普拉斯)](https://zhuanlan.zhihu.com/p/67522582)  


In [2]:
I = np.eye(6)
D = np.eye(6)
for i,j in enumerate([2,3,2,3,3,1]):
    D[i,i]=j
D

array([[2., 0., 0., 0., 0., 0.],
       [0., 3., 0., 0., 0., 0.],
       [0., 0., 2., 0., 0., 0.],
       [0., 0., 0., 3., 0., 0.],
       [0., 0., 0., 0., 3., 0.],
       [0., 0., 0., 0., 0., 1.]])

In [3]:
A = np.zeros((6,6))
A[[0,0,1,1,1,2,2,3,3,3,4,4,4,5],[1,4,0,2,4,1,3,2,4,5,0,1,3,3]]=1
A

array([[0., 1., 0., 0., 1., 0.],
       [1., 0., 1., 0., 1., 0.],
       [0., 1., 0., 1., 0., 0.],
       [0., 0., 1., 0., 1., 1.],
       [1., 1., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.]])

In [4]:
L=D-A
L

array([[ 2., -1.,  0.,  0., -1.,  0.],
       [-1.,  3., -1.,  0., -1.,  0.],
       [ 0., -1.,  2., -1.,  0.,  0.],
       [ 0.,  0., -1.,  3., -1., -1.],
       [-1., -1.,  0., -1.,  3.,  0.],
       [ 0.,  0.,  0., -1.,  0.,  1.]])

In [5]:
D_sym = LA.inv(D**(0.5)) # D**(-0.5)这种写法不对

D_sym
0.57735027*0.70710678
D_sym@L@D_sym

array([[ 0.71,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.58,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.71,  0.  ,  0.  ,  0.  ],
       [-0.  , -0.  , -0.  ,  0.58, -0.  , -0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.58,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  ]])

array([[ 1.  , -0.41,  0.  ,  0.  , -0.41,  0.  ],
       [-0.41,  1.  , -0.41,  0.  , -0.33,  0.  ],
       [ 0.  , -0.41,  1.  , -0.41,  0.  ,  0.  ],
       [ 0.  ,  0.  , -0.41,  1.  , -0.33, -0.58],
       [-0.41, -0.33,  0.  , -0.33,  1.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  , -0.58,  0.  ,  1.  ]])

In [11]:
I-D_sym@A@D_sym

array([[ 1.        , -0.40824829,  0.        ,  0.        , -0.40824829,
         0.        ],
       [-0.40824829,  1.        , -0.40824829,  0.        , -0.33333333,
         0.        ],
       [ 0.        , -0.40824829,  1.        , -0.40824829,  0.        ,
         0.        ],
       [ 0.        ,  0.        , -0.40824829,  1.        , -0.33333333,
        -0.57735027],
       [-0.40824829, -0.33333333,  0.        , -0.33333333,  1.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        , -0.57735027,  0.        ,
         1.        ]])

In [6]:
Lrw = LA.inv(D)@L
Lrw
I - LA.inv(D)@A

array([[ 1.  , -0.5 ,  0.  ,  0.  , -0.5 ,  0.  ],
       [-0.33,  1.  , -0.33,  0.  , -0.33,  0.  ],
       [ 0.  , -0.5 ,  1.  , -0.5 ,  0.  ,  0.  ],
       [ 0.  ,  0.  , -0.33,  1.  , -0.33, -0.33],
       [-0.33, -0.33,  0.  , -0.33,  1.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  , -1.  ,  0.  ,  1.  ]])

array([[ 1.  , -0.5 ,  0.  ,  0.  , -0.5 ,  0.  ],
       [-0.33,  1.  , -0.33,  0.  , -0.33,  0.  ],
       [ 0.  , -0.5 ,  1.  , -0.5 ,  0.  ,  0.  ],
       [ 0.  ,  0.  , -0.33,  1.  , -0.33, -0.33],
       [-0.33, -0.33,  0.  , -0.33,  1.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  , -1.  ,  0.  ,  1.  ]])

In [49]:
eva,eve = LA.eig(L)
eva
eve
np.sqrt(np.sum(eve**2,axis=1))
eve[:,0]@eve[:,2].T
np.round(eve[:,0]@eve[:,2].T,2)
# eve.shape
inner_prod_arr=np.ones(eve.shape,dtype='f4')

for i in range(eve.shape[0]):
    for j in range(eve.shape[1]):
        inner_prod_arr[i,j]=np.round(eve[:,i]@eve[:,j].T,2)
inner_prod_arr
        

array([-0.  ,  0.72,  1.68,  3.  ,  3.7 ,  4.89])

array([[-0.41, -0.41, -0.51,  0.29, -0.57, -0.03],
       [-0.41, -0.31,  0.04,  0.29,  0.66, -0.47],
       [-0.41, -0.07,  0.76,  0.29, -0.21,  0.36],
       [-0.41,  0.22,  0.2 , -0.58, -0.31, -0.56],
       [-0.41, -0.22, -0.2 , -0.58,  0.31,  0.56],
       [-0.41,  0.79, -0.29,  0.29,  0.11,  0.14]])

array([1., 1., 1., 1., 1., 1.])

array([[ 1.,  0.,  0., -0.,  0., -0.],
       [ 0.,  1., -0., -0.,  0.,  0.],
       [ 0., -0.,  1.,  0., -0., -0.],
       [-0., -0.,  0.,  1.,  0.,  0.],
       [ 0.,  0., -0.,  0.,  1.,  0.],
       [-0.,  0., -0.,  0.,  0.,  1.]], dtype=float32)

In [9]:
X=np.random.randint(0,2,size=(6,6))
X

array([[0, 0, 1, 0, 0, 1],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 1, 1],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 1, 0],
       [0, 1, 0, 1, 0, 0]])

In [17]:
x=np.random.randint(0,2,size=(6,1))
x

array([[1],
       [0],
       [1],
       [1],
       [0],
       [0]])

In [18]:
L@x

array([[ 2.],
       [-2.],
       [ 1.],
       [ 2.],
       [-2.],
       [-1.]])

In [19]:
x.T@L@x

array([[5.]])

In [11]:
L

array([[ 2., -1.,  0.,  0., -1.,  0.],
       [-1.,  3., -1.,  0., -1.,  0.],
       [ 0., -1.,  2., -1.,  0.,  0.],
       [ 0.,  0., -1.,  3., -1., -1.],
       [-1., -1.,  0., -1.,  3.,  0.],
       [ 0.,  0.,  0., -1.,  0.,  1.]])

In [10]:
L@X

array([[ 0.,  0.,  1., -1., -1.,  2.],
       [ 0.,  0., -2.,  2., -2., -2.],
       [-1.,  0.,  0.,  1.,  2.,  2.],
       [ 3., -1., -1., -2., -2., -1.],
       [-1.,  0.,  2., -1.,  3., -1.],
       [-1.,  1.,  0.,  1.,  0.,  0.]])

In [32]:
np.diag(-np.sort(-eva))
np.diag(np.sort(eva)[::-1])

array([[ 4.89,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  3.7 ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  3.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  1.68,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.72,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  , -0.  ]])

array([[ 4.89,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  3.7 ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  3.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  1.68,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.72,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  , -0.  ]])

In [31]:
np.sort(eva)[::-1]

array([ 4.89,  3.7 ,  3.  ,  1.68,  0.72, -0.  ])

In [22]:
help(np.sort)

Help on function sort in module numpy:

sort(a, axis=-1, kind=None, order=None)
    Return a sorted copy of an array.
    
    Parameters
    ----------
    a : array_like
        Array to be sorted.
    axis : int or None, optional
        Axis along which to sort. If None, the array is flattened before
        sorting. The default is -1, which sorts along the last axis.
    kind : {'quicksort', 'mergesort', 'heapsort', 'stable'}, optional
        Sorting algorithm. The default is 'quicksort'. Note that both 'stable'
        and 'mergesort' use timsort or radix sort under the covers and, in general,
        the actual implementation will vary with data type. The 'mergesort' option
        is retained for backwards compatibility.
    
        .. versionchanged:: 1.15.0.
           The 'stable' option was added.
    
    order : str or list of str, optional
        When `a` is an array with fields defined, this argument specifies
        which fields to compare first, second, etc.  A si

$对于（无向）图G及其具有特征值的拉普拉斯矩阵L：\lambda_0\leq \lambda_1\leq ... \leq \lambda_n \\
* L是对称的。\\
* L是半正定的(即 [公式] \\
* L是一个M矩阵\\
* L的每一行和列总和为零$

In [27]:
LA.inv(D)@L

array([[ 1.  , -0.5 ,  0.  ,  0.  , -0.5 ,  0.  ],
       [-0.33,  1.  , -0.33,  0.  , -0.33,  0.  ],
       [ 0.  , -0.5 ,  1.  , -0.5 ,  0.  ,  0.  ],
       [ 0.  ,  0.  , -0.33,  1.  , -0.33, -0.33],
       [-0.33, -0.33,  0.  , -0.33,  1.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  , -1.  ,  0.  ,  1.  ]])

In [35]:
LA.eig(L)

(array([-0.  ,  0.72,  1.68,  3.  ,  3.7 ,  4.89]),
 array([[-0.41, -0.41, -0.51,  0.29, -0.57, -0.03],
        [-0.41, -0.31,  0.04,  0.29,  0.66, -0.47],
        [-0.41, -0.07,  0.76,  0.29, -0.21,  0.36],
        [-0.41,  0.22,  0.2 , -0.58, -0.31, -0.56],
        [-0.41, -0.22, -0.2 , -0.58,  0.31,  0.56],
        [-0.41,  0.79, -0.29,  0.29,  0.11,  0.14]]))

## 傅里叶变换

### 连续傅里叶变换  
$\hat{f}(\xi)=\int_{-\infty}^{\infty} f(x) e^{-2 \pi i x \xi} d x,\ \xi为任意实数$  
$f(x)=\int_{-\infty}^{\infty} \hat{f}(\xi) e^{2 \pi i \xi x} d \xi,\ x为任意实数$  
### 离散傅里叶变换  
$x_{n}=\sum_{k=0}^{N-1} X_{k} e^{-i \frac{2 \pi}{N} k n} \quad n=0, \dots, N-1$

## [图卷积网络(GCN)新手村完全指南](https://zhuanlan.zhihu.com/p/54505069)

$f : \mathbb{R} \rightarrow \mathbb{C}$

## 稀疏矩阵

In [5]:
from scipy.sparse import csr_matrix
# create dense matrix
A = np.array([[1, 0, 0, 1, 0, 0], [0, 0, 2, 0, 0, 1], [0, 0, 0, 2, 0, 0]])
print(A)
# convert to sparse matrix (CSR method)
S = csr_matrix(A)
print(S)
# reconstruct dense matrix
B = S.todense()
print(B)

[[1 0 0 1 0 0]
 [0 0 2 0 0 1]
 [0 0 0 2 0 0]]
  (0, 0)	1
  (0, 3)	1
  (1, 2)	2
  (1, 5)	1
  (2, 3)	2
[[1 0 0 1 0 0]
 [0 0 2 0 0 1]
 [0 0 0 2 0 0]]


# [深度学习时代的图模型，清华发文综述图网络](https://zhuanlan.zhihu.com/p/53072284)

# *[一文读懂图卷积GCN](https://zhuanlan.zhihu.com/p/89503068)

# *[图卷积神经网络：Graph Convolutional Networks](https://zhuanlan.zhihu.com/p/91573076)   


In [24]:
IFrame('https://arxiv.org/pdf/1312.6203.pdf', width=600, height=300)

# *[图卷积网络到底怎么做，这是一份极简的Numpy实现](https://mp.weixin.qq.com/s/sg9O761F0KHAmCPOfMW_kQ)

In [50]:
A = np.matrix([
    [0, 1, 0, 0],
    [0, 0, 1, 1], 
    [0, 1, 0, 0],
    [1, 0, 1, 0]],
    dtype=float
)
A

matrix([[0., 1., 0., 0.],
        [0., 0., 1., 1.],
        [0., 1., 0., 0.],
        [1., 0., 1., 0.]])

In [51]:
X = np.matrix([
            [i, -i]
            for i in range(A.shape[0])
        ], dtype=float)
X

matrix([[ 0.,  0.],
        [ 1., -1.],
        [ 2., -2.],
        [ 3., -3.]])

In [52]:
A * X

matrix([[ 1., -1.],
        [ 5., -5.],
        [ 1., -1.],
        [ 2., -2.]])

In [53]:
I = np.matrix(np.eye(A.shape[0]))
I

matrix([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])

In [55]:
A_hat = A + I
A_hat
A_hat * X

matrix([[1., 1., 0., 0.],
        [0., 1., 1., 1.],
        [0., 1., 1., 0.],
        [1., 0., 1., 1.]])

matrix([[ 1., -1.],
        [ 6., -6.],
        [ 3., -3.],
        [ 5., -5.]])

In [66]:
A
np.sum(A, axis=1)
D = np.array(np.sum(A, axis=1)).reshape(-1)
D
D = np.matrix(np.diag(D))
D

matrix([[0., 1., 0., 0.],
        [0., 0., 1., 1.],
        [0., 1., 0., 0.],
        [1., 0., 1., 0.]])

matrix([[1.],
        [2.],
        [1.],
        [2.]])

array([1., 2., 1., 2.])

matrix([[1., 0., 0., 0.],
        [0., 2., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 2.]])

In [67]:
A
D**-1 * A

matrix([[0., 1., 0., 0.],
        [0., 0., 1., 1.],
        [0., 1., 0., 0.],
        [1., 0., 1., 0.]])

matrix([[0. , 1. , 0. , 0. ],
        [0. , 0. , 0.5, 0.5],
        [0. , 1. , 0. , 0. ],
        [0.5, 0. , 0.5, 0. ]])

In [69]:
D_hat = np.array(np.sum(A_hat, axis=1)).reshape(-1)
D_hat = np.matrix(np.diag(D_hat))
D_hat

matrix([[2., 0., 0., 0.],
        [0., 3., 0., 0.],
        [0., 0., 2., 0.],
        [0., 0., 0., 3.]])

In [72]:
W = np.matrix([[1, -1], [-1, 1]])
W
D_hat**-1 * A_hat * X * W

matrix([[ 1, -1],
        [-1,  1]])

matrix([[ 1.  , -1.  ],
        [ 4.  , -4.  ],
        [ 3.  , -3.  ],
        [ 3.33, -3.33]])

In [71]:
W = np.matrix([
             [1],
             [-1]
         ])
D_hat**-1 * A_hat * X * W

matrix([[1.  ],
        [4.  ],
        [3.  ],
        [3.33]])

In [78]:
from scipy.sparse import coo_matrix, csr_matrix, csc_matrix
row = np.array([0, 2, 2, 0, 1, 2])
col = np.array([0, 0, 1, 2, 2, 2])
data = np.array([1, 2, 3, 4, 5, 6])
csc_matrix((data, (row, col)), shape=(3, 3)).toarray()
coo_matrix((data, (row, col)), shape=(3, 3)).toarray()

array([[1, 0, 4],
       [0, 0, 5],
       [2, 3, 6]], dtype=int32)

array([[1, 0, 4],
       [0, 0, 5],
       [2, 3, 6]])

In [77]:
indptr = np.array([0, 2, 3, 6])
indices = np.array([0, 2, 2, 0, 1, 2])
data = np.array([1, 2, 3, 4, 5, 6])
csc_matrix((data, indices, indptr), shape=(3, 3)).toarray()


array([[1, 0, 4],
       [0, 0, 5],
       [2, 3, 6]])

# [拉普拉斯矩阵与拉普拉斯算子的关系](https://zhuanlan.zhihu.com/p/85287578) 

## [谱聚类方法推导和对拉普拉斯矩阵的理解](https://zhuanlan.zhihu.com/p/81502804)

## [【其实贼简单】拉普拉斯算子和拉普拉斯矩阵](https://zhuanlan.zhihu.com/p/67336297)

In [26]:
k_g = np.array([[-5,-3,0,0,0,0],
              [5,0,-7,0,0,-2],
              [0,3,0,1,0,0],
              [0,0,7,-1,-4,0],
              [0,0,0,0,4,2]])
K_G = pd.DataFrame(data=k_g, index=['A','B','C','D','E'],columns=['ab','ac','bd','dc','de','be'])
K_G

,ab,ac,bd,dc,de,be
A,-5,-3,0,0,0,0
B,5,0,-7,0,0,-2
C,0,3,0,1,0,0
D,0,0,7,-1,-4,0
E,0,0,0,0,4,2


In [50]:
f_g=np.array([3,5,-2,1,-5]).reshape(-1,1)
F_G = pd.DataFrame(data=f_g,index=['A','B','C','D','E'],columns=['FGv'])
F_G

,FGv
A,3
B,5
C,-2
D,1
E,-5


In [52]:
dgfg = k_g.T@f_g
dgfg

array([[ 10],
       [-15],
       [-28],
       [ -3],
       [-24],
       [-20]])

In [70]:
k_g_ = k_g.copy()
k_g_[k_g_>0]=1
k_g_[k_g_<0]=-1
k_g_
L = k_g_@k_g.T
L
k_g_@dgfg
f_g.T@k_g_@dgfg
f_g.T@k_g_@k_g.T@f_g

array([[-1, -1,  0,  0,  0,  0],
       [ 1,  0, -1,  0,  0, -1],
       [ 0,  1,  0,  1,  0,  0],
       [ 0,  0,  1, -1, -1,  0],
       [ 0,  0,  0,  0,  1,  1]])

array([[ 8, -5, -3,  0,  0],
       [-5, 14,  0, -7, -2],
       [-3,  0,  4, -1,  0],
       [ 0, -7, -1, 12, -4],
       [ 0, -2,  0, -4,  6]])

array([[  5],
       [ 58],
       [-18],
       [ -1],
       [-44]])

array([[560]])

array([[560]])

In [71]:
k_g_.shape,k_g.shape
d2 = k_g_@k_g_.T
d2

array([[ 2, -1, -1,  0,  0],
       [-1,  3,  0, -1, -1],
       [-1,  0,  2, -1,  0],
       [ 0, -1, -1,  3, -1],
       [ 0, -1,  0, -1,  2]])

In [77]:
a1 = L@f_g
a1
np.sqrt((a1**2).sum())
f_g.T@L@f_g

array([[  5],
       [ 58],
       [-18],
       [ -1],
       [-44]])

array([[560]])

In [88]:
f_g.T
d2
f_g
f_g.T@d2@f_g

array([[ 3,  5, -2,  1, -5]])

array([[ 2, -1, -1,  0,  0],
       [-1,  3,  0, -1, -1],
       [-1,  0,  2, -1,  0],
       [ 0, -1, -1,  3, -1],
       [ 0, -1,  0, -1,  2]])

array([[ 3],
       [ 5],
       [-2],
       [ 1],
       [-5]])

array([[190]])

In [100]:
n = 0
f_g
for i in L:
    (i*f_g.T).sum()
#     n+=(i*f_g.T).sum()**2
#     i*f_g.T*f_g.T
#     n+=np.abs(i*f_g.T*f_g.T).sum()
#     (i*f_g.T*f_g.T)**2
#     n+=(i*f_g.T*f_g.T).sum()
n
3*5+58*5+18*2-1+44*5


array([[ 3],
       [ 5],
       [-2],
       [ 1],
       [-5]])

5

58

-18

-1

-44

### [（二十四）用二阶微分（拉普拉斯算子）实现图像锐化](https://zhuanlan.zhihu.com/p/79548457)

In [38]:
L1 = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]])
L2 = np.array([[1, 1, 1], [1, -8, 1], [1, 1, 1]])
L1
L2

array([[ 0,  1,  0],
       [ 1, -4,  1],
       [ 0,  1,  0]])

array([[ 1,  1,  1],
       [ 1, -8,  1],
       [ 1,  1,  1]])

In [48]:
A1 = np.ones(shape=(3,3))
A1 = np.arange(9).reshape(3,3)
A1 = np.random.randint(0,2,size=(3,3))
A1
L1*A1
L2*A1
(L1*A1).sum()
(L2*A1).sum()

array([[1, 1, 0],
       [1, 1, 1],
       [1, 0, 0]])

array([[ 0,  1,  0],
       [ 1, -4,  1],
       [ 0,  0,  0]])

array([[ 1,  1,  0],
       [ 1, -8,  1],
       [ 1,  0,  0]])

-1

-3

# [如何理解 Graph Convolutional Network（GCN）](https://www.zhihu.com/question/54504471/answer/332657604)

In [105]:
import torchvision

ImportError: cannot import name 'PILLOW_VERSION' from 'PIL' (D:\MySoft\Anaconda3\lib\site-packages\PIL\__init__.py)

# 傅里叶变换

[傅里叶变换、拉普拉斯变换、Z 变换的联系是什么？为什么要进行这些变换？](https://www.zhihu.com/question/22085329)

# other


[漫谈GNN（一）](https://zhuanlan.zhihu.com/p/130827381)

# end

In [104]:
a = Piecewise((x,x>0),(0,True))
a
print_latex(a)

⎧x  for x > 0
⎨            
⎩0  otherwise

\begin{cases} x & \text{for}\: x > 0 \\0 & \text{otherwise} \end{cases}


$\begin{cases} x & \text{for}\: x > 0 \\0 & \text{otherwise} \end{cases}$